# Module

In [1]:

import os
os.environ["KERAS_BACKEND"] = "tensorflow"  # "jax" or "tensorflow" or "torch" 

import keras_cv
import keras
import keras.backend as K
import tensorflow as tf
import tensorflow_io as tfio

import numpy as np 
import pandas as pd

from glob import glob
from tqdm import tqdm

import librosa
import IPython.display as ipd
import librosa.display as lid

import matplotlib.pyplot as plt
import matplotlib as mpl

cmap = mpl.cm.get_cmap('coolwarm')

2024-06-07 15:04:02.406406: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-07 15:04:02.406554: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-07 15:04:02.553362: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/tmp/ipykernel_33/4180528519.py:23: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = mpl.cm.get_cmap('coolwarm')


## Config & Set seeds

In [2]:
class CFG:
    seed = 42
    
    # Input image size and batch size
    img_size = [128, 384]
    
    # Audio duration, sample rate, and length
    duration = 15 # second
    sample_rate = 32000
    audio_len = duration*sample_rate
    
    # STFT parameters
    nfft = 2028
    window = 2048
    hop_length = audio_len // (img_size[1] - 1)
    fmin = 20
    fmax = 16000
    
    # Number of epochs, model name
    preset = 'yolo_v8_s_backbone_coco'

    # Class Labels for BirdCLEF 24
    class_names = sorted(os.listdir('/kaggle/input/birdclef-2024/train_audio/'))
    num_classes = len(class_names)
    class_labels = list(range(num_classes))
    label2name = dict(zip(class_labels, class_names))
    name2label = {v:k for k,v in label2name.items()}

In [4]:
tf.keras.utils.set_random_seed(CFG.seed)

# Dataset Path 📁

In [5]:
BASE_PATH = '/kaggle/input/birdclef-2024'

# Test Data 📖

In [17]:
test_paths = glob(f'{BASE_PATH}/test_soundscapes/*ogg')
# During commit use `unlabeled` data as there is no `test` data.
# During submission `test` data will automatically be populated.
if len(test_paths)==0:
    test_paths = glob(f'{BASE_PATH}/unlabeled_soundscapes/*ogg')[:10]
test_df = pd.DataFrame(test_paths, columns=['filepath'])
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filepath  10 non-null     object
dtypes: object(1)
memory usage: 208.0+ bytes


# Modeling 🤖

Note that our model was trained on `10 second` duration audio files, but we will infer on `5-second` audio files (as per competition rules). To facilitate this, we have set the model input shape to `(None, None, 3)`, which will allow us to have variable-length input during training and inference.

In [7]:
def get_model(shape = (None, None, 3), weights = "imagenet"):
    model = keras_cv.models.ResNet50Backbone(input_shape = shape)
    flatten = tf.keras.layers.GlobalAveragePooling2D()(model.output)
    drop_out = tf.keras.layers.Dropout(0.5)(flatten)
    dense = tf.keras.layers.Dense(2048, activation = "relu")(drop_out)
    prediction = tf.keras.layers.Dense(182, activation = "softmax", name = "prediction")(dense)
    model = tf.keras.Model(model.input, prediction)
    return model


In [8]:
# Create an input layer for the model
inp = keras.layers.Input(shape=(None, None, 3))
# Pretrained backbone
backbone = keras_cv.models.ResNetBackbone.from_preset("resnet50_imagenet")
out = keras_cv.models.ImageClassifier(
    backbone=backbone,
    num_classes=CFG.num_classes,
    name="classifier"
)(inp)
# Build model
model = get_model()
# Load weights of trained model
model.load_weights("/kaggle/input/weight3/model1weight.h5")# Load weights of trained model
# Compile model with optimizer, loss and metrics
model.compile(optimizer="adam",
              loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.02),
              metrics=[keras.metrics.AUC(name='auc')],
             )

Attaching 'config.json' from model 'keras/resnetv1/keras/resnet50_imagenet/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/resnetv1/keras/resnet50_imagenet/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/resnetv1/keras/resnet50_imagenet/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/resnetv1/keras/resnet50/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/resnetv1/keras/resnet50/2' to your Kaggle notebook...


# Data Loader 🍚

The following code will decode the raw audio from `.ogg` file and also decode the spectrogram from the `audio` file. Additionally, we will apply Z-Score standardization and Min-Max normalization to ensure consistent inputs to the model.

In [10]:
# Decodes Audio
def build_decoder(with_labels=True, dim=1024):
    def get_audio(filepath):
        file_bytes = tf.io.read_file(filepath)
        audio = tfio.audio.decode_vorbis(file_bytes) # decode .ogg file
        audio = tf.cast(audio, tf.float32)
        if tf.shape(audio)[1]>1: # stereo -> mono
            audio = audio[...,0:1]
        audio = tf.squeeze(audio, axis=-1)
        return audio
    
    def create_frames(audio, duration=5, sr=32000):
        frame_size = int(duration * sr)
        audio = tf.pad(audio[..., None], [[0, tf.shape(audio)[0] % frame_size], [0, 0]]) # pad the end
        audio = tf.squeeze(audio) # remove extra dimension added for padding
        frames = tf.reshape(audio, [-1, frame_size]) # shape: [num_frames, frame_size]
        return frames
    
    def apply_preproc(spec):
        # Standardize
        mean = tf.math.reduce_mean(spec)
        std = tf.math.reduce_std(spec)
        spec = tf.where(tf.math.equal(std, 0), spec - mean, (spec - mean) / std)

        # Normalize using Min-Max
        min_val = tf.math.reduce_min(spec)
        max_val = tf.math.reduce_max(spec)
        spec = tf.where(tf.math.equal(max_val - min_val, 0), spec - min_val,
                              (spec - min_val) / (max_val - min_val))
        return spec

    def decode(path):
        # Load audio file
        audio = get_audio(path)
        # Split audio file into frames with each having 5 seecond duration
        audio = create_frames(audio)
        # Convert audio to spectrogram
        spec = keras.layers.MelSpectrogram(num_mel_bins=CFG.img_size[0],
                                             fft_length=CFG.nfft, 
                                              sequence_stride=CFG.hop_length, 
                                              sampling_rate=CFG.sample_rate)(audio)
        # Apply normalization and standardization
        spec = apply_preproc(spec)
        # Covnert spectrogram to 3 channel image (for imagenet)
        spec = tf.tile(spec[..., None], [1, 1, 1, 3])
        return spec
    
    return decode

In [11]:
# Build data loader
def build_dataset(paths, batch_size=1, decode_fn=None, cache=False):
    if decode_fn is None:
        decode_fn = build_decoder(dim=CFG.audio_len) # decoder
    AUTO = tf.data.experimental.AUTOTUNE
    slices = (paths,)
    ds = tf.data.Dataset.from_tensor_slices(slices)
    ds = ds.map(decode_fn, num_parallel_calls=AUTO) # decode audio to spectrograms then create frames
    ds = ds.cache() if cache else ds # cache files
    ds = ds.batch(batch_size, drop_remainder=False) # create batches
    ds = ds.prefetch(AUTO)
    return ds

# Inference 🏃

In [12]:
# Initialize empty list to store ids
ids = []

# Initialize empty array to store predictions
preds = np.empty(shape=(0, CFG.num_classes), dtype='float32')

# Build test dataset
test_paths = test_df.filepath.tolist()
test_ds = build_dataset(paths=test_paths, batch_size=1)

# Iterate over each audio file in the test dataset
for idx, specs in enumerate(tqdm(iter(test_ds), desc='test ', total=len(test_df))):
    print(idx)
    # Extract the filename without the extension
    filename = test_paths[idx].split('/')[-1].replace('.ogg','')
    
    # Convert to backend-specific tensor while excluding extra dimension
    specs = keras.ops.convert_to_tensor(specs[0])
    
    # Predict bird species for all frames in a recording using all trained models
    frame_preds = model.predict(specs, verbose=0)
    
    # Create a ID for each frame in a recording using the filename and frame number
    frame_ids = [f'{filename}_{(frame_id+1)*5}' for frame_id in range(len(frame_preds))]
    
    # Concatenate the ids
    ids += frame_ids
    # Concatenate the predictions
    preds = np.concatenate([preds, frame_preds], axis=0)
    
print(len(test_df))

test :   0%|          | 0/10 [00:00<?, ?it/s]

0


test :  10%|█         | 1/10 [00:13<02:02, 13.60s/it]

1


test :  20%|██        | 2/10 [00:15<00:52,  6.62s/it]

...

8


test :  90%|█████████ | 9/10 [00:29<00:02,  2.16s/it]

9


test : 100%|██████████| 10/10 [00:31<00:00,  3.17s/it]

10


In [13]:
print(len(test_ds))
print(len(test_paths))
print(len(preds[0]))
print(len(frame_preds))
print(len(test_ds))
print(filename)
print(frame_preds)

10
10
182
48
10
33854460
[[4.3259771e-03 4.0125921e-03 5.9785869e-04 ... 6.5644927e-05
  3.5749565e-04 1.6486569e-03]
 [4.1918731e-03 4.3139779e-03 8.9821802e-04 ... 4.6543377e-05
  1.2979540e-04 5.7664239e-03]
 [1.7168240e-03 4.3637143e-03 6.6714507e-04 ... 4.1100579e-05
  5.7036439e-05 5.5719921e-03]
 ...
 [1.6207107e-03 6.6549270e-03 2.9835599e-03 ... 9.1214525e-03
  5.7264412e-04 6.8518142e-03]
 [8.9442666e-04 1.1624815e-02 6.7122201e-03 ... 2.0252755e-02
  1.8562897e-04 6.2564667e-03]
 [1.4737559e-03 5.9950105e-03 1.5035543e-03 ... 3.0525874e-03
  7.0310535e-04 3.5798596e-03]]


In [14]:
test_ds

<_PrefetchDataset element_spec=TensorSpec(shape=(None, None, 128, 128, 3), dtype=tf.float32, name=None)>

# Submission ✉️

In [15]:
# Submit prediction
pred_df = pd.DataFrame(ids, columns=['row_id'])
print(pred_df)
pred_df.loc[:, CFG.class_names] = preds
pred_df.to_csv('submission.csv',index=False)
pred_df

            row_id
0     1872382287_5
1    1872382287_10
2    1872382287_15
3    1872382287_20
4    1872382287_25
..             ...
475   33854460_220
476   33854460_225
477   33854460_230
478   33854460_235
479   33854460_240

[480 rows x 1 columns]


,row_id,asbfly,ashdro1,ashpri1,ashwoo2,asikoe2,asiope1,aspfly1,aspswi1,barfly1,...,whbwoo2,whcbar1,whiter2,whrmun,whtkin2,woosan,wynlau1,yebbab1,yebbul3,zitcis1
0,1872382287_5,0.003150,0.011602,0.001392,0.000196,0.012220,0.001658,0.000424,0.007918,0.016402,...,0.013865,0.005775,0.000419,0.002555,0.061925,0.002699,0.000389,0.003282,0.000611,0.008459
1,1872382287_10,0.002887,0.012427,0.001102,0.000259,0.014886,0.001769,0.000648,0.007084,0.009867,...,0.016335,0.003189,0.000502,0.002253,0.060976,0.001889,0.000511,0.003467,0.001094,0.004401
2,1872382287_15,0.003115,0.012657,0.001085,0.000142,0.011752,0.001342,0.000404,0.007736,0.014555,...,0.014694,0.004223,0.000319,0.002027,0.065576,0.002115,0.000327,0.003313,0.000607,0.007034
3,1872382287_20,0.002890,0.013887,0.001529,0.000293,0.011494,0.001952,0.000724,0.007607,0.012552,...,0.014926,0.003652,0.000619,0.003058,0.056523,0.002623,0.000628,0.003541,0.000836,0.005902
4,1872382287_25,0.002426,0.009226,0.000649,0.000080,0.010208,0.001289,0.000301,0.006214,0.013643,...,0.017207,0.002817,0.000224,0.001299,0.056389,0.001430,0.000285,0.003410,0.000732,0.003591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,33854460_220,0.001355,0.007143,0.001851,0.000729,0.037000,0.001073,0.000083,0.015923,0.006438,...,0.004283,0.010909,0.000317,0.002898,0.043216,0.006473,0.000171,0.003884,0.000430,0.004475
476,33854460_225,0.001523,0.012712,0.002616,0.001082,0.027083,0.000877,0.000193,0.029938,0.007200,...,0.004437,0.007749,0.000309,0.004402,0.030272,0.007221,0.000253,0.005548,0.000347,0.002279
477,33854460_230,0.001621,0.006655,0.002984,0.001230,0.039136,0.001658,0.000126,0.020042,0.008760,...,0.003299,0.013803,0.000712,0.004596,0.027517,0.015718,0.000373,0.009121,0.000573,0.006852
478,33854460_235,0.000894,0.011625,0.006712,0.001830,0.011956,0.000985,0.000118,0.029391,0.007129,...,0.001130,0.002817,0.001478,0.012153,0.027218,0.054701,0.000443,0.020253,0.000186,0.006256
